In [15]:
import shutil, requests, os, glob
import subprocess

In [2]:
def download_file(url):
    local_filename = os.path.join('data/california-pam/',url.split('/')[-1])
    with requests.get(url, stream=True) as r:
        with open(local_filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

    return local_filename

In [10]:
# download vessel data for first 20 days
days = [f'{day:02d}' for day in range(1, 21)]
print(days)
urls = [f"https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_01_{day}.zip" for day in days]

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']


In [11]:
print(urls)

['https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_01_01.zip', 'https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_01_02.zip', 'https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_01_03.zip', 'https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_01_04.zip', 'https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_01_05.zip', 'https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_01_06.zip', 'https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_01_07.zip', 'https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_01_08.zip', 'https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_01_09.zip', 'https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_01_10.zip', 'https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_01_11.zip', 'https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_01_12.zip', 'https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_01_13.zip'

In [12]:
for url in urls:
    download_file(url)

In [16]:
# unzip files
for file in glob.glob('data/california-pam/AIS_2021*.zip'):
    subprocess.Popen(['unzip', file, '-d', 'data/vessel'])

Archive:  data/california-pam/AIS_2021_01_19.zip
  inflating: data/vessel/AIS_2021_01_19.csv  Archive:  data/california-pam/AIS_2021_01_20.zip
  inflating: data/vessel/AIS_2021_01_20.csv  Archive:  data/california-pam/AIS_2021_01_07.zip
  inflating: data/vessel/AIS_2021_01_07.csv  Archive:  data/california-pam/AIS_2021_01_15.zip
  inflating: data/vessel/AIS_2021_01_15.csv  Archive:  data/california-pam/AIS_2021_01_14.zip
  inflating: data/vessel/AIS_2021_01_14.csv  Archive:  data/california-pam/AIS_2021_01_01.zip
  inflating: data/vessel/AIS_2021_01_01.csv  Archive:  data/california-pam/AIS_2021_01_11.zip
  inflating: data/vessel/AIS_2021_01_11.csv  Archive:  data/california-pam/AIS_2021_01_05.zip
  inflating: data/vessel/AIS_2021_01_05.csv  Archive:  data/california-pam/AIS_2021_01_03.zip
  inflating: data/vessel/AIS_2021_01_03.csv  Archive:  data/california-pam/AIS_2021_01_12.zip
  inflating: data/vessel/AIS_2021_01_12.csv  Archive:  data/california-pam/AIS_2021_01_17.zip
  inflating

In [4]:
! pip install geopandas fiona shapely

Defaulting to user installation because normal site-packages is not writeable


# Read files

Attempt to count vessel activity within area at around 0AM

In [17]:
import geopandas as gpd
import pandas as pd

In [18]:
ais_0101 = pd.read_csv('data/vessel/AIS_2021_01_01.csv')

In [20]:
len(ais_0101)

6624812

In [21]:
ais_0101.head()

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TranscieverClass
0,338325166,2021-01-01T00:00:20,28.08771,-82.77474,0.0,0.0,511.0,SALT WIFE,IMO0000000,NaN,37.0,NaN,13.0,6.0,NaN,NaN,B
1,368163740,2021-01-01T00:00:18,26.10727,-80.12551,0.0,360.0,511.0,TOTALLY EXCELLENT,IMO0000000,WDL7818,37.0,NaN,18.0,4.0,NaN,NaN,B
2,366811570,2021-01-01T00:00:23,30.66150,-88.03264,0.0,196.0,270.0,NATALIE H,IMO7200245,WDA4637,31.0,0.0,18.0,7.0,NaN,NaN,A
3,368140750,2021-01-01T00:00:24,30.23140,-93.25791,0.0,50.5,272.0,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,A
4,368170010,2021-01-01T00:00:28,29.57223,-90.59391,0.0,316.2,197.0,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,A


In [26]:
# our audio data is around 12AM
query_time = "BaseDateTime > '2021-01-01T00:00:00' & BaseDateTime < '2021-01-01T01:00:00'"

In [45]:
ais_0101_timed = ais_0101.query(query_time)

In [52]:
len(ais_0101_timed)

211081

Get bounding box within hydrophone area to count number of vessels:
36.713°N, 122.186°W

122.186W = -122.186

-> Choose bounding box within 1 degrees:

In [47]:
lon = -122.186
lat = 36.713 

In [56]:
query_spatial = f"LAT > {lat-0.25} & LAT < {lat + 0.25} & LON > {lon-0.25} & LON < {lon+0.25}"

In [57]:
ais_0101_res = ais_0101_timed.query(query_spatial)

In [58]:
len(ais_0101_res) # 319 within bounding box

0

In [59]:
ais_0101_res

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TranscieverClass


In [60]:
ais_0101_res['MMSI'].unique()

array([], dtype=int64)

In [63]:
# develop function to output number of ships
def get_vessel_activity(data_path):
    # data path has date, so we can keep track of time
    date = os.path.basename(data_path)[4:-4]
    
    df = pd.read_csv(data_path, engine='pyarrow')
    query_time = df[df['BaseDateTime'].dt.hour == 0]
    query_boundingbox = f"LAT > {lat-0.25} & LAT < {lat + 0.25} & LON > {lon-0.25} & LON < {lon+0.25}"
    res = query_time.query(query_boundingbox)
    print(f"{date} has {len(res)} within bounding box")
    return res

In [64]:
results = {}
for f in glob.glob('data/vessel/AIS*.csv'):
    res =get_vessel_activity(f)
    results[f] = res

2021_01_08 has 12 within bounding box
2021_01_19 has 0 within bounding box
2021_01_01 has 0 within bounding box
2021_01_03 has 55 within bounding box
2021_01_15 has 0 within bounding box
2021_01_17 has 0 within bounding box
2021_01_14 has 0 within bounding box
2021_01_18 has 72 within bounding box
2021_01_04 has 40 within bounding box
2021_01_13 has 0 within bounding box
2021_01_11 has 0 within bounding box
2021_01_02 has 42 within bounding box
2021_01_06 has 42 within bounding box
2021_01_07 has 0 within bounding box
2021_01_09 has 0 within bounding box
2021_01_12 has 0 within bounding box
2021_01_16 has 37 within bounding box
2021_01_20 has 47 within bounding box
2021_01_05 has 53 within bounding box
2021_01_10 has 69 within bounding box


In [65]:
for d, res in results.items():
    date = os.path.basename(d)[4:-4]
    unique_vessels = len(res['MMSI'].unique())
    print(f"{date} has {unique_vessels} vessels within bounding box")

2021_01_08 has 1 vessels within bounding box
2021_01_19 has 0 vessels within bounding box
2021_01_01 has 0 vessels within bounding box
2021_01_03 has 2 vessels within bounding box
2021_01_15 has 0 vessels within bounding box
2021_01_17 has 0 vessels within bounding box
2021_01_14 has 0 vessels within bounding box
2021_01_18 has 2 vessels within bounding box
2021_01_04 has 1 vessels within bounding box
2021_01_13 has 0 vessels within bounding box
2021_01_11 has 0 vessels within bounding box
2021_01_02 has 2 vessels within bounding box
2021_01_06 has 1 vessels within bounding box
2021_01_07 has 0 vessels within bounding box
2021_01_09 has 0 vessels within bounding box
2021_01_12 has 0 vessels within bounding box
2021_01_16 has 1 vessels within bounding box
2021_01_20 has 1 vessels within bounding box
2021_01_05 has 1 vessels within bounding box
2021_01_10 has 2 vessels within bounding box


In [67]:
import pickle

with open('data/vessel/vessels_count.pickle', 'wb') as f:
    pickle.dump(results, f)